## Set paths, seed for random stuffs, etc

In [ ]:
import os 
import numpy as np
import pandas as pd
TEST_DATASET_PATH = os.environ.get("PATH_TEST_DATASETS", "/kaggle/input/plant-seedlings-classification/test")

### Kaggle path

In [ ]:
# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = os.environ.get("PATH_DATASETS", "/kaggle/input/plant-seedlings-classification")
TRAIN_DATASET_PATH = os.environ.get("PATH_TRAIN_DATASETS", "/kaggle/input/plant-seedlings-classification/train")
TEST_DATASET_PATH = os.environ.get("PATH_TEST_DATASETS", "/kaggle/input/plant-seedlings-classification/test")

# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "/kaggle/working")
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

num_workers = os.cpu_count()

### Google Colab path

In [ ]:

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = os.environ.get("PATH_DATASETS", "/kaggle/input/plant-seedlings-classification")
TRAIN_DATASET_PATH = os.environ.get("PATH_TRAIN_DATASETS", "/kaggle/input/plant-seedlings-classification/train")
TEST_DATASET_PATH = os.environ.get("PATH_TEST_DATASETS", "/kaggle/input/plant-seedlings-classification/test")

# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "./saved_models/")
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

num_workers = os.cpu_count()

### Local Repo path

In [ ]:
# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = os.environ.get("PATH_DATASETS", "./data")
TRAIN_DATASET_PATH = os.environ.get("PATH_TRAIN_DATASETS", "./data/train")
TEST_DATASET_PATH = os.environ.get("PATH_TEST_DATASETS", "./data/test")

# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "./saved_models/")
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

num_workers = int(os.cpu_count()/ 2)

In [ ]:
plt.set_cmap("cividis")
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg", "pdf")  # For export
matplotlib.rcParams["lines.linewidth"] = 2.0
sns.reset_orig()

# Setting the seed
seed = 13
L.seed_everything(seed)
# Ensure that all operations are deterministic on GPU (if used) for reproducibility\
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

#os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)


In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

## Create submission file

In [ ]:
import os 
import numpy as np
import pandas as pd
TEST_DATASET_PATH = os.environ.get("PATH_TEST_DATASETS", "/kaggle/input/plant-seedlings-classification/test")

In [ ]:
species = ["Charklock"] * len(os.listdir(TEST_DATASET_PATH))

In [ ]:
submission = pd.DataFrame({"file": os.listdir(TEST_DATASET_PATH),
                         "species": species})

submission.to_csv("submission.csv", index=False, header=True)

In [ ]:
os.listdir("/kaggle/working")